In [7]:
import cv2
import numpy as np

from PIL import Image
# import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

In [8]:
video_path= 'video.hevc'
frames = list()

# Load the video
video = cv2.VideoCapture(video_path)

while True:
  # Read the next frame
  ret, frame = video.read()
  if not ret:
    break

  frame = cv2.resize(frame, (512, 512))
  # Convert image in BGR format to RGB.
  frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
  # Add a batch dimension which is required by the model.
  frame = np.expand_dims(frame, axis=0)/255.0

  frames.append(frame)
  if len(frames) == 800:
    break

frames = np.array(frames)
frames.shape

OpenCV: Couldn't read video stream from file "video.hevc"
[ERROR:0@74.428] global cap.cpp:166 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.7.0) /Users/xperience/GHA-OCV-Python/_work/opencv-python/opencv-python/opencv/modules/videoio/src/cap_images.cpp:253: error: (-5:Bad argument) CAP_IMAGES: can't find starting number (in the name of file): video.hevc in function 'icvExtractPattern'




In [ ]:
model_url =  'https://tfhub.dev/google/HRNet/camvid-hrnetv2-w48/1'
seg_model = hub.load(model_url)


In [13]:
# Function to convert a single channel mask representation to an RGB mask.
def class_to_rgb(mask_class):

    # Create RGB channels
    r_map = np.zeros_like(mask_class).astype(np.uint8)
    g_map = np.zeros_like(mask_class).astype(np.uint8)
    b_map = np.zeros_like(mask_class).astype(np.uint8)


    class_id=17
    index = mask_class == class_id

    if class_id == 17:
      r_map[index] = 128
      g_map[index] = 64
      b_map[index] = 128

    seg_map_rgb = np.stack([r_map, g_map, b_map], axis=2)

    return seg_map_rgb

In [9]:

def image_overlay(image, seg_map_rgb):


    image = (image*255.0).astype(np.uint8)


    image = cv2.addWeighted(image, 1.0, cv2.cvtColor(cv2.cvtColor(seg_map_rgb, cv2.COLOR_RGB2BGR), cv2.COLOR_RGB2BGR), 0.6, 0.0)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    return image

In [5]:
def segment(images, model):
    overlayed_images = list()

    for img in images:

        # Forward pass through the model (convert the tensor output to a numpy array).
        pred_mask = model.predict(img).numpy()

        # Remove the background class added by the model.
        pred_mask = pred_mask[:,:,:,1:]

        # Remove the batch dimension.
        pred_mask = np.squeeze(pred_mask)
        pred_mask_class = np.argmax(pred_mask, axis=-1)

        # Convert the predicted (class) segmentation map to a color segmentation map.
        pred_mask_rgb = class_to_rgb(pred_mask_class)

        fig = plt.figure(figsize=(20, 15))

        # Display the original image.
        ax1 = fig.add_subplot(1,3,1)
        ax1.imshow(img[0])
        ax1.title.set_text('Input Image')
        plt.axis('off')

        # Display the predicted color segmentation mask.
        ax2 = fig.add_subplot(1,3,2)
        ax2.set_title('Predicted Mask')
        ax2.imshow(pred_mask_rgb)
        plt.axis('off')

        # Display the predicted color segmentation mask overlayed on the original image.
        overlayed_image = image_overlay(img[0], pred_mask_rgb)
        ax4 = fig.add_subplot(1,3,3)
        ax4.set_title('Overlayed Image')
        ax4.imshow(overlayed_image)
        plt.axis('off')

        plt.show()
        overlayed_images.append(overlayed_image)
    return overlayed_images

In [ ]:
images = segment(frames, seg_model)

In [ ]:
# Specify the desired video format and frame rate
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = 20

# Create a VideoWriter object
video_writer = cv2.VideoWriter('output.avi', fourcc, fps, (images[0].shape[1], images[0].shape[0]))

# Iterate through the list of images
for image in images:
    # Write the frame to the video
    video_writer.write(image)

# Close the VideoWriter object
video_writer.release()

print('Video saved as output.avi')